In [1]:
import pandas as pd
import numpy as np
import sys
import csv
import os
params = {}
##数据集文件夹路径
params['data_path']=r'E:\文件夹\大三下\移动互联网课设\大作业\CWRU0408\data'
###数据清洗后文件夹路径
params['iso_path']=r'E:\文件夹\大三下\移动互联网课设\大作业\CWRU0408\isolation'
###数据归一化后文件夹路径
params['normal_path']=r'E:\文件夹\大三下\移动互联网课设\大作业\CWRU0408\normalization'
###三种特征提取方法csv文件路径，实际要用到的是time_fre_feature（时频域特征提取）
params['time_fre_feature_path']=r'E:\文件夹\大三下\移动互联网课设\大作业\CWRU0408\feature\time_fre.csv'
# params['time_feature_path']=r'E:\文件夹\大三下\移动互联网课设\大作业\CWRU0319\feature\time.csv'
# params['fre_feature_path']=r'E:\文件夹\大三下\移动互联网课设\大作业\CWRU0319\feature\fre.csv'

In [2]:
#########数据清洗
#####1
def data_delete(params):
    with open(params['path'],'r') as f:
    #1.创建阅读器对象
        reader = csv.reader(f)
    #2.读取文件第一行数据
        head_row=next(reader)
    data_attribute = []
    for item in head_row:
        data_attribute.append(item)

 #读取数据并删除最后一列标签
    tn = pd.read_csv(params['path']) 
    tn.dropna(inplace=True)
    train = np.array(tn)
    train_x = train[:, :-1]

#存标签
    train_y = train[:,-1]
    train_y = np.array(train_y)

#对所有数据行进行异常检测
    train_x = np.array(train_x)
    clf = IsolationForest(n_estimators=params['n_estimators'], 
                      max_samples=params['max_samples'], 
                      contamination=params['contamination'], 
                      max_features=params['max_features'], 
                      bootstrap=False, n_jobs=1, random_state=None, 
                      verbose=0).fit(train_x)
#pred存入的是每一行数据的预测值，是1或者-1
    pred = clf.predict(train_x)
    normal = train_x[pred == 1]
    abnormal = train_x[pred == -1]

#删除pred为-1的行数据
    df = pd.DataFrame(pd.read_csv(params['path']))[0:pred.size]
    df['pred']=pred

#data2=data1[-data1.sorce.isin([61])]
    df2 = df[-df.pred.isin([-1])]
    df2 = df2.drop(['pred'],axis=1)

#将清洗之后的数据存入csv文件
    data_out = df2.iloc[:,:].values
    csvfile2 = open(params['opath'],'w',newline='')
    writer = csv.writer(csvfile2)
    writer.writerow(data_attribute)   #存属性
    m = len(data_out)
    for i in range(m):
        writer.writerow(data_out[i])

In [3]:
####数据归一化
######2

def normalization(params):
    with open(params['path'],'r') as f:
    #1.创建阅读器对象
        reader = csv.reader(f)
    #2.读取文件第一行数据
        head_row=next(reader)
    data_attribute = []
    for item in head_row:
        data_attribute.append(item)

    #读取数据并删除最后一列标签
    tn = pd.read_csv(params['path']) 
    tn.dropna(inplace=True)
    train = np.array(tn)
    train_x = train[:, :-1]

    train_y = train[:,-1]
    train_y = np.array(train_y)

    #对所有数据行进行标准化
    min_max_scaler = preprocessing.MinMaxScaler(feature_range=params['feature_range'])

    X_train_minmax = min_max_scaler.fit_transform(train_x)

    out=np.column_stack((X_train_minmax,train_y))

    csvfile2 = open(params['opath'],'w',newline='')
    writer = csv.writer(csvfile2)
    writer.writerow(data_attribute)   #存属性
    m = len(out)

    for i in range(m):
         writer.writerow(out[i])

In [4]:
####时频域特征提取

def  one_row(arr,label_mean,LAbel):
      result_list = []
      arr_add=arr.loc[:,:]
      for j in range(int(params['wave_win']/params['len_piece'])):  
             arr_add =arr_add.append(arr,ignore_index=True)
      for i in arr_add.columns:    
        cD=wavedec(arr_add[i],'db10',level=params['wave_layer'])   
        for i in range(params['wave_layer']+1):
             ener_cD = np.square(cD[i]).sum()   
             list_para = [ener_cD]
             result_list.extend(list_para)
      list_label=[label_mean.mean()]
      result_list.extend(list_label)
      result_list.append(LAbel)
      return result_list

def time_fre_feature(params,Label):
    df_data = pd.read_csv(params['path'])
    data=np.array(df_data)
    lenth=data[:,-1]
    lenth=len(np.array(lenth))
# ##测试集不加窗，一个文件提取一排特征
#     params['len_piece']=lenth
# ##训练集加窗，提升准确率
    i=1
    data_out=[]
    while i*params['len_piece']<lenth:
        arr_pic=df_data.loc[(i-1)*params['len_piece']:i*params['len_piece']-1,:params['head_label2']]
        label_pic=df_data.loc[(i-1)*params['len_piece']:i*params['len_piece']-1,params['head_label']]
        i=i+1
        data_out.append(one_row(arr_pic,label_pic,Label))
    if params['len_piece']>lenth:                
        arr_pic=df_data.loc[:lenth,:params['head_label2']]
        label_pic=df_data.loc[:lenth,params['head_label']]
    else:                                                        
        arr_pic=df_data.loc[lenth-params['len_piece']:lenth,:params['head_label2']]
        label_pic=df_data.loc[lenth-params['len_piece']:lenth,params['head_label']]
    data_out.append(one_row(arr_pic,label_pic,Label))
    return data_out

In [5]:
##数据清洗
from sklearn.ensemble import IsolationForest
params['n_estimators'] = 100
params['max_samples'] ='auto'
params['contamination'] = 0.1
params['max_features'] = 1.0

filelists = os.listdir(params['data_path'])
for info in filelists:
    params['path']= os.path.join(params['data_path'],info)
    params['opath'] =os.path.join(params['iso_path'],info)
    data_delete(params)

In [6]:
##数据归一化
from sklearn import preprocessing 
params['feature_range'] = (0,1)

filelists = os.listdir(params['iso_path'])
for info in filelists:
    params['path']= os.path.join(params['iso_path'],info)
    params['opath'] =os.path.join(params['normal_path'],info)
    normalization(params)

In [7]:
###时频域特征提取
from pywt import wavedec
import math
params['len_piece']=100    
params['wave_layer']=4      
params['wave_win']=38*pow(2,params['wave_layer']-1)-1  

filelists = os.listdir(params['normal_path'])

##加头
with open(os.path.join(params['normal_path'],filelists[1]),'r') as f:
    reader = csv.reader(f)
    head_row=next(reader)
    params['head_label']=head_row[-1]
    head_row=head_row[:-1] 
    params['head_label2']=head_row[-1]
data_head=[]
head_out=[]
for i in head_row:   
     head_out=[i+'_cA'+str(params['wave_layer'])]
     data_head.extend(head_out)
     for j in range(params['wave_layer']):
            head_out=[i+'_cD'+str(params['wave_layer']-j)]
            data_head.extend(head_out)
headlabel_out=[params['head_label']]
data_head.extend(headlabel_out)
data_head.append('label')
head_out=[]
head_out.append(data_head)
##写入标题
pd.DataFrame(head_out).to_csv(params['time_fre_feature_path'],index=False,header=False)
###首先设定测试集时的初始label值,顺便给每个文件打个标签
label=4
for info in filelists:
    ###给特征添加label
    if 'B'in info:
        label=1
    elif 'IR'in info:
        label=3
    elif 'NORMAL'in info:
        label=0
    elif 'OR'in info:
        label=2
    else:###即测试集状态
        label+=1
    params['path']= os.path.join(params['normal_path'],info)
    #写入数据
    pd.DataFrame(time_fre_feature(params,label)).to_csv(params['time_fre_feature_path'],mode='a',index=False,header=False)